In [1]:
import pandas as pd

# Load the CSV file
file_path = r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\IDS-DRR-Assam-Risk-Model\RiskScoreModel\data\risk_score_final_district.csv"
print("Loading CSV file from:", file_path)
df = pd.read_csv(file_path)
print("CSV file loaded successfully. Shape:", df.shape)

Loading CSV file from: D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\IDS-DRR-Assam-Risk-Model\RiskScoreModel\data\risk_score_final_district.csv
CSV file loaded successfully. Shape: (10320, 100)


In [2]:
# Selecting only numeric columns along with 'object-id', 'timeperiod', and 'financial-year'
numeric_columns = df.select_dtypes(include=["number"]).columns
list(numeric_columns).remove("rc-area")
numeric_columns = numeric_columns.drop("rc-area")
df_numeric = df[["object-id", "timeperiod", "financial-year"] + list(numeric_columns)]

# Identifying columns that are not included
excluded_columns = [col for col in df.columns if col not in df_numeric.columns]
print("Excluded columns:", excluded_columns)

df_numeric.head()

Excluded columns: ['district', 'rc-area', 'revenue-ci', 'dtname', 'dtcode11']


,object-id,timeperiod,financial-year,total-tender-awarded-value,sopd-tenders-awarded-value,sdrf-sanctions-awarded-value,sdrf-tenders-awarded-value,ridf-tenders-awarded-value,ltif-tenders-awarded-value,cidf-tenders-awarded-value,...,efficiency,total-tender-awarded-value-fy-cumsum,sdrf-sanctions-awarded-value-fy-cumsum,sdrf-tenders-awarded-value-fy-cumsum,preparedness-measures-tenders-awarded-value-fy-cumsum,immediate-measures-tenders-awarded-value-fy-cumsum,others-tenders-awarded-value-fy-cumsum,topsis-score,risk-score,total-infrastructure-damage
0,18-758-00257,2021_04,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.829466,0.0,0.0,0.0,0.0,0.0,0.0,0.889910,5,0.0
1,18-756-00249,2021_04,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.784425,0.0,0.0,0.0,0.0,0.0,0.0,0.749157,5,0.0
2,18-303-00121,2021_04,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.769380,0.0,0.0,0.0,0.0,0.0,0.0,0.749157,5,0.0
3,18-320-00206,2021_04,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.831167,0.0,0.0,0.0,0.0,0.0,0.0,0.744425,5,0.0
4,18-316-00185,2021_04,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.843519,0.0,0.0,0.0,0.0,0.0,0.0,0.732745,4,0.0


In [4]:
df_melted = df_numeric.melt(id_vars=["object-id", "timeperiod", "financial-year"], var_name="factor", value_name="score")
print("Data melted. Shape:", df_melted.shape)

df_melted.head()

Data melted. Shape: (949440, 5)


,object-id,timeperiod,financial-year,factor,score
0,18-758-00257,2021_04,2021-2022,total-tender-awarded-value,0.0
1,18-756-00249,2021_04,2021-2022,total-tender-awarded-value,0.0
2,18-303-00121,2021_04,2021-2022,total-tender-awarded-value,0.0
3,18-320-00206,2021_04,2021-2022,total-tender-awarded-value,0.0
4,18-316-00185,2021_04,2021-2022,total-tender-awarded-value,0.0


In [5]:

df_transposed = df_melted.pivot(index=["factor", "timeperiod", "financial-year"], columns="object-id", values="score").reset_index()
print("Data pivoted. Shape:", df_transposed.shape)
df_transposed.head()
# Save or display the transformed data
output_file = "Transformed_Assam_Data.csv"
df_transposed.to_csv(output_file, index=False)
print("Transformed data saved to:", output_file)

Data pivoted. Shape: (4416, 218)
Transformed data saved to: Transformed_Assam_Data.csv


In [7]:
# Verify with the source data. Given factor, district, timeperiod and object-id, the score should match

factor = "sdrf-sanctions-awarded-value"
district = '18-319-00202'
timeperiod = '2024_07'
df_transposed.head()
print("modified")
print(df_transposed[(df_transposed["factor"] == factor) & (df_transposed["timeperiod"] == timeperiod) ][[district, "timeperiod", "financial-year", "factor"]])
print("original")
df[(df["object-id"] == district) & (df["timeperiod"] == timeperiod)][['timeperiod', 'financial-year', factor]]



modified
object-id  18-319-00202 timeperiod financial-year  \
2874                0.0    2024_07      2024-2025   

object-id                        factor  
2874       sdrf-sanctions-awarded-value  
original


,timeperiod,financial-year,sdrf-sanctions-awarded-value
7138,2024_07,2024-2025,0.0


In [7]:
dff = pd.read_csv(r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\IDS-DRR-Assam-Risk-Model\RiskScoreModel\data\Transformed_Assam_Data.csv")
conditions = []
operator_map = {
            '==': lambda col, val: col == val,
            '!=': lambda col, val: col != val,
            '>': lambda col, val: col > val,
            '<': lambda col, val: col < val,
            '>=': lambda col, val: col >= val,
            '<=': lambda col, val: col <= val,
            'in': lambda col, val: col.isin(val),
            'not in': lambda col, val: ~col.isin(val)
        }
conditions.append(operator_map["=="](dff["factor"], "sdrf-sanctions-awarded-value"))

dff = dff[pd.concat(conditions, axis=1).all(axis=1)]
dff

,factor,timeperiod,financial-year,18-300,18-300-00101,18-300-00102,18-300-00103,18-300-00104,18-300-00105,18-300-00106,...,18-799,18-799-00124,18-799-00125,18-799-00265,18-816,18-816-00235,18-816-00236,18-816-00258,18-816-00259,18-816-00261
3360,sdrf-sanctions-awarded-value,2021_04,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3361,sdrf-sanctions-awarded-value,2021_05,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3362,sdrf-sanctions-awarded-value,2021_06,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3363,sdrf-sanctions-awarded-value,2021_07,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3364,sdrf-sanctions-awarded-value,2021_08,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3365,sdrf-sanctions-awarded-value,2021_09,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3366,sdrf-sanctions-awarded-value,2021_10,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3367,sdrf-sanctions-awarded-value,2021_11,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3368,sdrf-sanctions-awarded-value,2021_12,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3369,sdrf-sanctions-awarded-value,2022_01,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
